<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/CBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
event=pd.read_csv("eco_event.csv")
place=pd.read_csv("eco_place.csv")
rating=pd.read_csv("eco_rating.csv")

In [3]:
event

,event_id,event_img,event_name,event_place,event_date,event_about
0,1,https://storage.googleapis.com/travelee-capsto...,FESTIVAL ECOTOURISM 2022 - Membangun Investasi...,"Manokwari City Mall (MCM), Papua Barat",29 September - 1 Oktober 2022,Ayo bergabung dalam FESTIVAL ECOTOURISM 2022 R...
1,2,https://storage.googleapis.com/travelee-capsto...,Festival Taman nasional Taman Wisata Alam Indo...,"Lapangan Aldiron, Jl. Gatot Subroto, Jakarta",5 - 7 Agustus 2022,Sambut Festival FTNTWA dan Festival Ekowisata ...
2,3,https://storage.googleapis.com/travelee-capsto...,Medan - Belawan Mangrove Forest Festival,"Mako Lantamal-I, Medan, Sumatra Utara",19 Agustus 2018,Sambut keindahan hutan mangrove di Medan - Bel...
3,4,https://storage.googleapis.com/travelee-capsto...,FESTIVAL MANGROVE 2022,"Kuala Langsa, Aceh",10 Desember 2022,Selamat datang di Festival Mangrove 2022. Raya...
4,5,https://storage.googleapis.com/travelee-capsto...,INDOFEST INDONESIA OUTDOOR FESTIVAL 2023,"Istora Senayan, Jakarta Selatan",1 - 4 Juni 2023,Selamat datang di Indofest Indonesia Outdoor F...
5,6,https://storage.googleapis.com/travelee-capsto...,Indonesia Ecotourism Summit 2023 Ecotourism Fo...,"InterContinental Hotel, Bandung",23 Mei 2023,Selamat datang di Indonesia Ecotourism Summit ...


In [4]:
place

,place_id,place_name,place_description,category,city,price,rating,description_location,place_img,gallery_photo_img1,gallery_photo_img2,gallery_photo_img3,place_map
0,1,Taman Nasional Gunung Leuser,Taman Nasional Gunung Leuser adalah salah satu...,"Budaya,Taman Nasional",Aceh,"Rp25,000",4.5,"Barisan mountain range, Aceh 24653",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Taman+Nasio...
1,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,"Rp10,000",4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9
2,3,Desa Wisata Penglipuran,Desa Wisata Penglipuran adalah sebuah desa wis...,"Budaya,Desa Wisata",Bali,"Rp25,000",4.8,"Jl. Penglipuran, Kubu, Kec. Bangli, Kabupaten ...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Desa+Wisata...
3,4,Taman Nasional Bali Barat,Taman Nasional Bali Barat adalah kawasan konse...,"Taman Nasional,Cagar Alam",Bali,"Rp15,000",4.5,"Sumber Klampok, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Taman+Nasio...
4,5,Bukit Jamur,Bukit Jamur Ciwidey adalah satu dari sekian ba...,Cagar Alam,Bandung,"Rp12,000",4.2,"Sugihmukti, Kec. Pasirjambu, Kabupaten Bandung...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Bukit+Jamur
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,178,Studio Alam Gamplong,Studio Gamplong atau Studio Alam Gamplong adal...,Taman Hiburan,Yogyakarta,"Rp10,000",4.4,"Gamplong 1, Dukuh, Sumberrahayu, Kec. Moyudan,...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,NaN,https://www.google.com/maps/search/Studio+Alam...
178,179,Watu Goyang,Watu Goyang adalah sebuah fenomena geologis ya...,"Budaya,Cagar Alam",Yogyakarta,"Rp2,500",4.4,"Cempluk, Jl. Watu Goyang No.1, Griloyo, Mangun...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Watu+Goyang
179,180,Watu Lumbung,Letak Kampung Edukasi Watu Lumbung yang berada...,Cagar Alam,Yogyakarta,"Rp5,000",4.3,"Area Hutan, Balong, Kec. Girisubo, Kabupaten G...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Watu+Lumbung
180,181,Wisata Alam Kalibiru,Kalibiru sebenarnya adalah sebuah nama hutan y...,"Budaya,Cagar Alam",Yogyakarta,"Rp20,000",4.4,"Jalan Waduk Sermo, Kalibiru, Hargowilis, Kec. ...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Wisata+Alam...


In [5]:
rating

,user_id,place_id,user_rating
0,1,28,2
1,1,86,4
2,1,94,4
3,1,146,3
4,2,17,2
...,...,...,...
844,156,144,4
845,156,15,5
846,156,29,4
847,156,52,4
